In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
d = pd.read_csv("/kaggle/input/cassava-leaf-disease-classification/train.csv")

In [ ]:
d

In [ ]:
img_path = "/kaggle/input/cassava-leaf-disease-classification/train_images/"

# Checking types of disease and what are their label

In [ ]:
import json

with open("/kaggle/input/cassava-leaf-disease-classification/label_num_to_disease_map.json") as f:
    data = json.load(f)
print(data)

In [ ]:
from PIL import Image 
for i in d.head():
    img = Image.open("/kaggle/input/cassava-leaf-disease-classification/train_images/1235188286.jpg") 
    img.show() 

# Load Images

In [ ]:
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 
plt.figure(figsize=(8,5))
for ind in d.head().index:
    #plt.subplot(231)
    img = mpimg.imread(img_path + str(d["image_id"][ind]))
    plt.imshow(img)
    plt.show()    

# Check count of each label

In [ ]:
import seaborn as sns
sns.countplot(d["label"])

# Image Augmentation


|For deep learning models we need large amount of data. With the help of image augmentation we can expand the size of training dataset and can variety of already existing images. For this we will be using ImageDataGenerator from deep learning library keras. There are bunch of techniques for image augmentation.
* By using Horizontal flip,
* Vertical Flip,
* Width Shift,
* Height Shift,
* Rotation Augmentation,
* Zoom Augmentation.


In [ ]:
# For image augmentation when class_mode is categorical, y_col must be string/tuple/list.
d["label"] = d["label"].astype(str)

In [ ]:
# Split dataset for training and validation part.
from sklearn.model_selection import train_test_split

train,val = train_test_split(d, test_size = 0.2)


In [ ]:
#Image Augmentation
from keras.preprocessing.image import ImageDataGenerator

# Mostly we use augmentaion only for training dataset.
train_datagen = ImageDataGenerator(width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   cval = 0.,
                                   #rotation_range=35,
                                   #rescale=1./255,
                                   shear_range=0.3,
                                   zoom_range=0.3,
                                   horizontal_flip=True)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(train,
                                                    directory = img_path,
                                                    x_col = "image_id",
                                                    y_col = "label",
                                                    target_size=(128, 128),
                                                    batch_size = 64,
                                                    class_mode = "categorical")

validation_generator = validation_datagen.flow_from_dataframe(val,
                                                        directory = img_path,
                                                        x_col = "image_id",
                                                        y_col = "label",
                                                        target_size=(128, 128),
                                                        batch_size=64,
                                                        class_mode = "categorical")


# Building Model

In [ ]:
#import important library
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
#Building very simple Convolution Network
def create_model():
    sequential_model = keras.Sequential(
    [
        keras.Input(shape=(250, 250, 3)),
        #First Convolution layer
        layers.Conv2D(32, (3,3), strides=2, activation="relu"),
        #second Convolution layer
        layers.Conv2D(64, (3,3), activation="relu"),
        #third convolution layer
        layers.Conv2D(64, (3,3), activation="relu"),
    ])
    sequential_model.compile()
    return sequential_model


In [ ]:
model = create_model()

In [ ]:
#Checking summary of model
model.summary()